In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from secrets import IEX_CLOUD_API_TOKEN

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 87340560, 'calculationPrice': 'close', 'change': -5.17, 'changePercent': -0.03106, 'close': 169.58, 'closeSource': 'fcaifiol', 'closeTime': 1709626932059, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 169.1, 'delayedPriceTime': 1655176615064, 'extendedChange': -0.24, 'extendedChangePercent': -0.00144, 'extendedPrice': 170.21, 'extendedPriceTime': 1684939378882, 'high': 171.74, 'highSource': 'Cosel', 'highTime': 1671620953543, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 166.01, 'iexCloseTime': 1669925114757, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 170.69, 'iexOpenTime': 1714289339096, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1679271976867, 'latestPrice': 172.05, 'latestSource': 'Close', 'latestTime': 'April 14, 2022', 'latestUpdate': 1682799197613, 'latestVolume': 79064807, 'low': 166.28, 'lowSource': 'eCsol', 'lowTime': 1710972236641, 

In [4]:
price = data['latestPrice']
market_cap = data['marketCap']

In [5]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [33]:
symbol_groups = list([stocks['Ticker'][x:x+100] for x in range(0, len(stocks['Ticker']), 100)])
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])    

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns
            ),
            ignore_index=True
        )
        except KeyError as e:
            pass
        
final_dataframe

/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_25371/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_25371/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_25371/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/2c/8ns7vywn32bcdsff75nn0nn09j2q4z/T/ipykernel_25371/430141924.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.66,38745319899,N/A
1,AAL,20,12680135623,N/A
2,AAP,225.68,13780983006,N/A
3,AAPL,168.22,2784630576930,N/A
4,ABBV,164.05,299228371884,N/A
...,...,...,...,...
496,YUM,125.16,36564724924,N/A
497,ZBH,127,26481910497,N/A
498,ZBRA,412.82,20898353163,N/A
499,ZION,65.67,9719332219,N/A


In [ ]:
# Calculating number of shares to buy